In [1]:
import os
import pandas as pd

# Set the working directory
os.chdir("C:\TradeGroup")
my_wd = os.getcwd()

file_path_Imp = f"{my_wd}/Data/RawData/Imp140208first.xlsx"
file_path_Exp = f"{my_wd}/Data/RawData/Exp140208first.xlsx"


In [6]:
def create_folder(folder_path):
    if not os.path.exists(f'{new_directory}/{folder_path}'):
        os.makedirs(f'{new_directory}/{folder_path}')
    else:
        print(f'Already Exists')

In [3]:
Imports = pd.read_excel(file_path_Imp,sheet_name="Stats",skiprows=1)
Exports = pd.read_excel(file_path_Exp,sheet_name="Stats",skiprows=1)


In [4]:
Imports = Imports.rename(columns={"سال":"Year","ماه":"Month","نام گمرک":"Custom",
                                  "نام کشور":"Country","کدتعرفه":"Tcode","شرح تعرفه": "Tdesc",
                                  "وزن (کیلوگرم)": "Weight_Imp", "ارزش (ریال)":"Rials_Imp","ارزش (دلار)": "Dollars_Imp"})

Exports = Exports.rename(columns={"سال":"Year","ماه":"Month","نام گمرک":"Custom",
                                  "نام کشور":"Country","کدتعرفه":"Tcode","شرح تعرفه": "Tdesc",
                                  "وزن (کیلوگرم)": "Weight_Exp", "ارزش (ریال)":"Rials_Exp","ارزش (دلار)": "Dollars_Exp"})

unique_months = Imports['Month'].unique()

In [7]:
new_directory = my_wd
create_folder('Data/Monthly_Data/Detailed')
DataFrames =[]
unique_months = unique_months[~pd.isna(unique_months)]
for month_value in unique_months:
    # Create a new DataFrame for each month
    df_name_Imp = f'Imports{int(month_value):02d}'
    df_name_Exp = f'Exports{int(month_value):02d}'
    df_name = f'M{int(month_value):02d}'
    
    # Filter rows for the current month
    globals()[df_name_Imp] = Imports[Imports['Month'] == month_value].reset_index(drop=True)
    globals()[df_name_Exp] = Exports[Exports['Month'] == month_value].reset_index(drop=True)
    globals()[df_name]     = pd.merge(globals()[df_name_Exp],globals()[df_name_Imp],
                                      on=['Year','Month','Custom','Country','Tcode','Tdesc'],
                                      how='outer')
    globals()[df_name]=globals()[df_name].fillna(0)
    DataFrames.append(df_name)
    xlsx_filename = os.path.join(my_wd, f'Data/Monthly_Data/Detailed/M1402{int(month_value):02d}Detailed.xlsx')
    globals()[df_name].to_excel(xlsx_filename, index=False)    
    


In [9]:
for df in DataFrames:
    M_monthly= globals()[df].groupby(['Tcode','Tdesc']).agg(
        Tcode=('Tcode','first'),
        Tdesc=('Tdesc','first'),
        Weight_Imp=('Weight_Imp','sum'),
        Rials_Imp=('Rials_Imp','sum'),
        Dollars_Imp=('Dollars_Imp','sum'),
          Weight_Exp=('Weight_Exp','sum'),
        Rials_Exp=('Rials_Exp','sum'),
        Dollars_Exp=('Dollars_Exp','sum')
    ).reset_index(drop=True)
    xlsx_filename_Sum = os.path.join(my_wd, f'Data/Monthly_Data//M1402{df}.xlsx')
    M_monthly.to_excel(xlsx_filename_Sum, index=False)

    